In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, accuracy_score, recall_score, precision_recall_curve, auc
from imblearn.over_sampling import SMOTE, ADASYN
import category_encoders as ce
import xgboost as xgb
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

import sys
import os
import pickle
import time

# Add the path to the project root directory to the python path
sys.path.append(os.path.abspath("C:/Users/mabid/OneDrive/Desktop/Projects - Ongoing/BNP Credit Card Fraud Detection/fraud_detection/"))

from src.features import build_features

In [32]:
MODEL_RANK = -1
EVAL_DATA = '../data/processed/model_validation_data.csv'

# Show models that are in the models directory that has "16" in its name
active_models = [model for model in os.listdir('../models/') if '16' in model]
print(active_models)
MODEL_PATH = active_models[MODEL_RANK]

['01-Anas-XGBTuned-16_Feb_2024_06_48_04.pickle', '01-Anas-XGBTuned-16_Feb_2024_06_52_44.pickle', '01-Anas-XGBTuned-16_Feb_2024_07_10_22.pickle', '01-Anas-XGBTuned-16_Feb_2024_07_12_25.pickle', '01-Anas-XGBTuned-16_Feb_2024_07_14_13.pickle']


In [34]:
# Load the model
with open(f'../models/{MODEL_PATH}', 'rb') as file:
    model_args = pickle.load(file)

In [36]:
# Unpack the model arguments
class Model():
    def __init__(self, model_args):
        self.model = model_args['model']
        self.pipe = model_args['pipe']
        self.model_name = model_args['model_name']
        self.prediction_matrix_type = model_args['prediction_matrix_type']
        self.y_true_train = model_args['y_true_train']
        self.y_pred_train = model_args['y_pred_train']
        self.y_true_test = model_args['y_true_test']
        self.y_pred_test = model_args['y_pred_test']
        self.y_pred_valid = None
        self.y_true_valid = None

model_XGB = Model(model_args)

# Load the evaluation data
eval_data = pd.read_csv(EVAL_DATA)

# Transform the evaluation data
eval_data = model_XGB.pipe.transform(eval_data)
if model_XGB.prediction_matrix_type == 'xgb.DMatrix':
    eval_data = xgb.DMatrix(eval_data)

C:\Users\mabid\AppData\Local\Temp\ipykernel_7816\3131462969.py:16: DtypeWarning: Columns (104,105,106,107,108,109,110,111,112,113,114,115,117,118,119,120) have mixed types. Specify dtype option on import or set low_memory=False.
  eval_data = pd.read_csv(EVAL_DATA)


In [26]:
model_XGB.y_pred_valid = model_XGB.model.predict(eval_data)

In [38]:
eval_data = pd.read_csv(EVAL_DATA)
eval_data

C:\Users\mabid\AppData\Local\Temp\ipykernel_7816\2534171464.py:1: DtypeWarning: Columns (104,105,106,107,108,109,110,111,112,113,114,115,117,118,119,120) have mixed types. Specify dtype option on import or set low_memory=False.
  eval_data = pd.read_csv(EVAL_DATA)


,ID,item1,item2,item3,item4,item5,item6,item7,item8,item9,...,Nbr_of_prod_purchas17,Nbr_of_prod_purchas18,Nbr_of_prod_purchas19,Nbr_of_prod_purchas20,Nbr_of_prod_purchas21,Nbr_of_prod_purchas22,Nbr_of_prod_purchas23,Nbr_of_prod_purchas24,Nb_of_items,fraud_flag
0,38100,TELEVISIONS HOME CINEMA,TELEVISIONS HOME CINEMA,AUDIO ACCESSORIES,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0
1,13409,BEDROOM FURNITURE,BEDROOM FURNITURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0
2,56447,TELEVISIONS HOME CINEMA,FULFILMENT CHARGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0
3,94627,COMPUTERS,COMPUTER PERIPHERALS ACCESSORIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0
4,11531,COMPUTERS,COMPUTER SOFTWARE,FULFILMENT CHARGE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18553,16785,"TELEPHONES, FAX MACHINES & TWO-WAY RADIOS",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
18554,11514,COMPUTER PERIPHERALS & ACCESSORIES,WARRANTY,AUDIO ACCESSORIES,WARRANTY,FULFILMENT CHARGE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,0
18555,26807,COMPUTERS,COMPUTER PERIPHERALS ACCESSORIES,BAGS CARRY CASES,FULFILMENT CHARGE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0
18556,84360,COMPUTERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
